## Skip_gram_with_negative_sampling

In [110]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter
flatten = lambda l: [item for sublist in l for item in sublist]

In [111]:
print(torch.__version__)
print(nltk.__version__)

1.0.0
3.3


In [112]:
FloatTensor =  torch.FloatTensor
LongTensor = torch.LongTensor
ByteTensor =  torch.ByteTensor

In [113]:
def getBatch(batch_size, train_data):
    random.shuffle(train_data)
    sindex = 0
    eindex = batch_size
    while eindex < len(train_data):
        batch = train_data[sindex: eindex]
        temp = eindex
        eindex = eindex + batch_size
        sindex = temp
        yield batch
    
    if eindex >= len(train_data):
        batch = train_data[sindex:]
        yield batch

In [114]:
def prepare_sequence(seq, word2index):
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return Variable(LongTensor(idxs))

def prepare_word(word, word2index):
    return Variable(LongTensor([word2index[word]]) if word2index.get(word) is not None else LongTensor([word2index["<UNK>"]]))

## Data load and Preprocessing

In [115]:
corpus = list(nltk.corpus.gutenberg.sents('melville-moby_dick.txt'))[:500]
corpus = [[word.lower() for word in sent] for sent in corpus]

In [116]:
word_count = Counter(flatten(corpus))

In [117]:
MIN_COUNT = 3
exclude = []

In [118]:
for w, c in word_count.items(): # 3번 아래로 나타나는 단어들은 제외한다. 
    if c < MIN_COUNT:
        exclude.append(w)

In [119]:
sparse_words = []               # 3번 아래로 등장하는 단어들
for w,c in word_count.items():
    sparse_words.append(w)
sparse_words[:10]

['[',
 'moby',
 'dick',
 'by',
 'herman',
 'melville',
 '1851',
 ']',
 'etymology',
 '.']

## Prepare train data

In [120]:
vocab = list(set(flatten(corpus)) - set(exclude))

In [121]:
word2index = {}
for vo in vocab:
    if word2index.get(vo) is None:
        word2index[vo] = len(word2index)
        
index2word = {v:k for k, v in word2index.items()}

In [122]:
WINDOW_SIZE = 5
windows =  flatten([list(nltk.ngrams(['<DUMMY>'] * WINDOW_SIZE + c + ['<DUMMY>'] * WINDOW_SIZE, WINDOW_SIZE * 2 + 1)) for c in corpus])

train_data = []

for window in windows:
    for i in range(WINDOW_SIZE * 2 + 1):
        if window[i] in exclude or window[WINDOW_SIZE] in exclude: 
            continue # min_count
        if i == WINDOW_SIZE or window[i] == '<DUMMY>': 
            continue
        train_data.append((window[WINDOW_SIZE], window[i]))

X_p = []
y_p = []

for tr in train_data:
    X_p.append(prepare_word(tr[0], word2index).view(1, -1))
    y_p.append(prepare_word(tr[1], word2index).view(1, -1))
    
train_data = list(zip(X_p, y_p))

In [123]:
len(train_data) # 자주등장하는 단어들 포함시켜서 이전에 skip_gram모델때보다 더 많음

50242

## Build Unigram Distribution ** 0.75

![](https://user-images.githubusercontent.com/36406676/54072330-bfc2a480-42bc-11e9-8759-d27c561d28d9.jpg)

In [124]:
Z = 0.001
word_count = Counter(flatten(corpus))
num_total_words = sum([c for w, c in word_count.items() if w not in exclude])

In [125]:
num_total_words

7798

In [126]:
unigram_table= []
for vo in vocab:
    unigram_table.extend([vo]*int(((word_count[vo]/num_total_words)**0.75)/Z))

In [127]:
word_count['city']

4

In [131]:
(4/7798)**0.75*1000

3.4084550450597506

In [134]:
unigram_table[:6]
## 3500중 coffin이 3개를 차지함 

['coffin', 'coffin', 'coffin', 'voyages', 'voyages', 'voyages']

In [161]:
print(len(vocab), len(unigram_table)) 

478 3500


In [162]:
random.choice(unigram_table) # 이런식으로 negative sampling 할 예정

'very'

## Negative_Sampling

In [171]:
def negative_sampling(targets, unigram_table, k):
    batch_size = targets.size(0)
    neg_samples = []
    for i in range(batch_size):
        nsample = []
        target_index = targets[i].data.cpu().tolist()[0]
        while len(nsample) < k: # num of sampling
            neg = random.choice(unigram_table)
            if word2index[neg] == target_index:
                continue
            nsample.append(neg)
        neg_samples.append(prepare_sequence(nsample, word2index).view(1,-1))
        
    return torch.cat(neg_samples) # B X K
        
        

![](https://user-images.githubusercontent.com/36406676/54081188-cf390080-4343-11e9-980e-812f74dcb1bb.jpg)

목적함수를 이해해보자
![](https://user-images.githubusercontent.com/36406676/54084068-6bc4c800-436f-11e9-9b4c-f5d300989194.jpg)

In [172]:
class SkipgramNegSampling(nn.Module):
    
    def __init__(self, vocab_size, projection_dim):
        super(SkipgramNegSampling, self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, projection_dim) # center embedding
        self.embedding_u = nn.Embedding(vocab_size, projection_dim) # out embedding
        self.logsigmoid = nn.LogSigmoid()
        
        initrange = (2.0 / (vocab_size + projection_dim)) ** 0.5 # Xavier init
        self.embedding_v.weight.data.uniform_(-initrange,initrange)
        self.embedding_u.weight.data.uniform_(-0.0,0.0) # init
        
    def forward(self, center_words, target_words, negative_words):
        center_embeds = self.embedding_v(center_words) # B X 1 X D
        target_embeds = self.embedding_u(target_words) # B X 1 X D
        # outer_embeds가 사라짐 더효율적인 계산위해 negative sampling
        
        
        neg_embeds = -self.embedding_u(negative_words) # B X K X D
        positive_score = target_embeds.bmm(center_embeds.transpose(1,2)).squeeze(2) # BX1XD * BXDX1 -> BX1X1 -> BX1
        negative_score = torch.sum(neg_embeds.bmm(center_embeds.transpose(1,2)).squeeze(2),1).view(negs.size(0),-1) # Batch별 sum
                                                                                                                    # BXK -> B -> BX1
        
        loss = self.logsigmoid(positive_score) + self.logsigmoid(negative_score)
        
        return -torch.mean(loss)
    
    def prediction(self, inputs):
        embeds = self.embedding_v(inputs)
        
        return embeds

## Train

In [173]:
EMBEDDING_SIZE = 30
BATCH_SIZE = 256
EPOCH = 100
NEG = 10 # NUM OF NEGATIVE SAMPLING

In [174]:
losses = []
model = SkipgramNegSampling(len(word2index), EMBEDDING_SIZE)
optimizer = optim.Adam(model.parameters(),lr = 0.001)

In [176]:
for epoch in range(EPOCH):
    for i, batch in enumerate(getBatch(BATCH_SIZE, train_data)):
        
        inputs, targets = zip(*batch)
        
        inputs = torch.cat(inputs)
        targets = torch.cat(targets)
        negs = negative_sampling(targets,unigram_table,NEG)
        model.zero_grad()
        
        loss = model(inputs, targets, negs)
        
        loss.backward()
        optimizer.step()
        
        losses.append(loss.data)
        
    if epoch % 10 ==0:
        print("Epoch:  %d, mean_loss : %.02f"%(epoch, np.mean(losses)))
        losses = []

Epoch:  0, mean_loss : 1.06
Epoch:  10, mean_loss : 0.86
Epoch:  20, mean_loss : 0.79
Epoch:  30, mean_loss : 0.74
Epoch:  40, mean_loss : 0.71
Epoch:  50, mean_loss : 0.69
Epoch:  60, mean_loss : 0.67
Epoch:  70, mean_loss : 0.65
Epoch:  80, mean_loss : 0.64
Epoch:  90, mean_loss : 0.63


## Test

In [181]:
def word_similarity(target, vocab):
    target_V = model.prediction(prepare_word(target, word2index))
    similarities = []
    for i in range(len(vocab)):
        if vocab[i] == target: 
            continue
        
        vector = model.prediction(prepare_word(list(vocab)[i], word2index))
        
        cosine_sim = F.cosine_similarity(target_V, vector).data.tolist()[0]
        similarities.append([vocab[i], cosine_sim])
    return sorted(similarities, key=lambda x: x[1], reverse=True)[:10]

In [182]:
test = random.choice(list(vocab))
test

'tell'

In [183]:
word_similarity(test, vocab)

[['why', 0.8125784397125244],
 ['cannot', 0.7415319681167603],
 ['does', 0.7253918051719666],
 ['particular', 0.6915906071662903],
 ['broiled', 0.663489818572998],
 [';--', 0.6374010443687439],
 ['hear', 0.6341304183006287],
 ['thinks', 0.6225622892379761],
 ['till', 0.6158430576324463],
 ['matter', 0.6151326894760132]]